# Measuring Australian Receiver Functions

Receiver functions are measured using the `hiperseis` package, which contains routines for downloading and processing waveforms on HPC systems.
This notebook details the work flow but does not actually run anything.
Example shell/job submission scripts are given for each stage.

# 1. Extracting Waveforms

This stage uses `hiperseis/seismic/extract_event_traces.py` to download waveforms from the web.  The following bash script downloads waveforms for a given seismic network.

```bash
#!/bin/bash

ROOT= # path to directory containing AusSedThick repo

# hiperseis is not currently properly installable, so needs to be added to PYTHONPATH
export HIPERSEIS=$ROOT/AusSedThickRF/hiperseis
export PYTHONPATH=$HIPERSEIS/:$PYTHONPATH

# Location of waveform database
WAVEFORM_DB='IRIS'

# Path to a STATIONXML file with details of all the stations you need.
# Can be created using obspy.
INVENTORY_FILE=

# Event selection criteria
EPICENTRAL_MIN=30
EPICENTRAL_MAX=95

# Output files
NET=$1
OUTDIR=$ROOT/AusSedThickRF/data/rf_data/$NET-analysis
mkdir -p $OUTDIR
EVENT_CAT=$OUTDIR/${NET}_events.xml
TRACES=$OUTDIR/${NET}_event_waveforms.h5

# Run
mpirun -n 8 python $HIPERSEIS/seismic/extract_event_traces.py \
    --waveform-database $WAVEFORM_DB \
    --inventory-file $INVENTORY_FILE \
    --event-catalog-file $EVENT_CAT \
    --event-trace-datafile $TRACES \
    --network-list $NET \
    --p-data \
    --distance-range $EPICENTRAL_MIN $EPICENTRAL_MAX \
    --resample-hz 50 \
    > $OUTDIR/extract_waveforms_${NET}.log
```

Below is the full list of options for `extract_event_traces.py`.

```
Usage: extract_event_traces.py [OPTIONS]

Options:
  --inventory-file FILE           Optional path to input inventory file
                                  corresponding to waveform source provided
                                  through, --waveform-database. Note that this
                                  parameter is required only when the waveform
                                  source is not a definition file for a
                                  FederatedASDFDataSet, in which case, the
                                  relevant inventory is extracted internally.
  --network-list TEXT             A space-separated list of networks (within
                                  quotes) to process.  [default: *]
  --station-list TEXT             A space-separated list of stations (within
                                  quotes) to process.  [default: *]
  --waveform-database TEXT        Location of waveform source database from
                                  which to extract traces. May be a recognized
                                  service provider from
                                  obspy.clients.fdsn.header.URL_MAPPINGS (e.g.
                                  "ISC"), an actual URL (e.g.
                                  "http://auspass.edu.au") or a file path. If
                                  detected as a URL, the obspy client
                                  get_waveform function will be used to
                                  retrieve waveforms from web service.
                                  Otherwise, if detected as a valid file path,
                                  then it must be the path to a definition
                                  file for a FederatedASDFDataSet, e.g. "/g/da
                                  ta/ha3/Passive/SHARED_DATA/Index/asdf_files.
                                  txt".  [required]
  --event-catalog-file FILE       Path to event catalog file, e.g.
                                  "catalog_7X_for_rf.xml". If file already
                                  exists, it will be loaded, otherwise it will
                                  be created by querying the ISC web service.
                                  Note that for traceability, start and end
                                  times will be appended to file name.
                                  [required]
  --event-trace-datafile FILE     Path to output file, e.g.
                                  "7X_event_waveforms.h5". Note that for
                                  traceability, start and end datetimes will
                                  be appended to file name.  [required]
  --start-time TEXT               Start datetime in ISO 8601 format, e.g.
                                  "2009-06-16T03:42:00". If empty, will be
                                  inferred from the inventory file.
  --end-time TEXT                 End datetime in ISO 8601 format, e.g.
                                  "2011-04-01T23:18:49". If empty, will be
                                  inferred from the inventory file.
  --taup-model TEXT               Theoretical tau-p Earth model to use for
                                  Trace stats computation. Other
                                  possibilities, such as ak135, are documented
                                  here: https://docs.obspy.org/packages/obspy.
                                  taup.html  [default: iasp91]
  --distance-range <FLOAT FLOAT>...
                                  Range of teleseismic distances (in degrees)
                                  to sample relative to the mean lat,lon
                                  location  [default: 0, 180.0]
  --magnitude-range <FLOAT FLOAT>...
                                  Range of seismic event magnitudes to sample
                                  from the event catalog for P/S arrivals.
                                  [default: 5.5, 10.0]
  --sw-magnitude-range <FLOAT FLOAT>...
                                  Range of seismic event magnitudes to sample
                                  from the event catalog for surface waves.
                                  [default: 6.0, 10.0]
  --catalog-only                  If set, only generate catalog file and exit.
                                  Used for preparing input file on HPC systems
                                  with no internet access.
  --resample-hz FLOAT             Resampling frequency (default 10 Hz) for
                                  output P/S traces  [default: 10]
  --sw-resample-hz FLOAT          Resampling frequency (default 2 Hz) for
                                  surface waves  [default: 2]
  --p-data                        Extracts waveform data around P-arrival
  --s-data                        Extracts waveform data around S-arrival
  --sw-data                       Extracts waveform data around surface-wave
                                  arrival
  --dry-run                       Reports events available to each station, by
                                  wave-type and exits without outputting any
                                  data. Has no effect on --catalog-only mode.
  --help                          Show this message and exit.
  ```

# 2. Measuring Receiver Functions

This stage measures receiver functions from the waveforms previously measured.
The main script is `hiperseis/seismic/receiver_rf/generate_rf.py`.

The configuration of the calculation is handled by a `config_rfs.json` file, an example of which is shown below.

```json
{
  "filtering": {
    "resample_rate": 50.0,
    "taper_limit": 0.05,
    "filter_band": [0.1, 1.0]
  },
  "processing": {
      "rotation_type": "ZRT",
      "deconv_domain": "iter",
      "normalize": true
  }
}
```

Full configuration options are:
```json
"filtering":  # Filtering settings
{
    "resample_rate": float # Resampling rate in Hz
    "taper_limit": float   # Fraction of signal to taper at end, between 0 and 0.5
    "filter_band": (float, float) # Filter pass band (Hz). Not required for freq-domain deconvolution.
    "channel_pattern": # Ordered list of preferred channels, e.g. 'HH*,BH*',
                        # where channel selection is ambiguous.
    "baz_range": (float, float) or [(float, float), ...] # Discrete ranges of source back azimuth to use (degrees).
        # Each value must be between 0 and 360. May be a pair or a list of pairs for multiple ranges.
}

"processing":  # RF processing settings
{
    "custom_preproc":
    {
    "import": 'import custom symbols',  # statement to import required symbols
    "func": 'preproc functor'  # expression to get handle to custom preprocessing functor
    "args": {}  # additional kwargs to pass to func
    }
    "trim_start_time": float # Trace trim start time in sec, relative to onset
    "trim_end_time": float # Trace trim end time in sec, relative to onset
    "rotation_type": str # Choice of ['zrt', 'lqt']. Rotational coordinate system
                        # for aligning ZNE trace components with incident wave direction
    "deconv_domain": str # Choice of ['time', 'freq', 'iter']. Whether to perform deconvolution
                        # in time or freq domain, or iterative technique
    "gauss_width": float # Gaussian freq domain filter width. Only required for freq-domain deconvolution
    "water_level": float # Water-level for freq domain spectrum. Only required for freq-domain deconvolution
    "spiking": float # Spiking factor (noise suppression), only required for time-domain deconvolution
    "normalize": bool # Whether to normalize RF amplitude
}

"correction": # corrections to be applied to data for named stations prior to RF computation
{
    "plot_dir": str # path to folder where plots related to orientation corrections are to be saved
    "swap_ne": list # list of NET.STA.LOC for which N and E channels are to be swapped, e.g ["OA.BL27."],
    "rotate": list # list of NET.STA.LOC that are to be rotated to maximize P-arrival energy on \
                    the primary RF component, e.g ["OA.BL27."]
    "negate": list # list of NET.STA.LOC.CHA that are to be negated, e.g ["OA.BL27..HHZ"]
}
```

The following bash script measures the receiver functions and performs some initial quality checks.

```bash
#!/bin/bash

ROOT= # path to directory containing AusSedThick repo

# hiperseis is not currently properly installable, so needs to be added to PYTHONPATH
export HIPERSEIS=$ROOT/AusSedThickRF/hiperseis
export PYTHONPATH=$HIPERSEIS/:$PYTHONPATH

NET=$1

DATADIR=$ROOT/data/rf_data/$NET-analysis # where the waveforms have been saved
INFILE=$(ls $DATADIR/*_event_waveforms_*.h5 | head -1) # picks the latest waveform file
NOW=$(date +'%Y%m%d_%H%M%S')
OUTDIR=$DATADIR/$NOW
CONFIG= # Path to config_rfs.json file

mkdir -p $OUTDIR
cp $CONFIG $OUTDIR

mpirun -n 8 python $HIPERSEIS/seismic/receiver_fn/generate_rf.py \
    $INFILE \
    $OUTDIR/${NET}_rf.h5 \
    --config-file $CONFIG \
    > $OUTDIR/compute_rfs.log 2>&1

python $HIPERSEIS/seismic/receiver_fn/rf_quality_filter.py \
    $OUTDIR/${NET}_rf.h5 \
    $OUTDIR/${NET}_rf_qc.h5 \
    --temp-dir $TMPDIR \
    --no-parallel \
    >> $OUTDIR/compute_rfs.log 2>&1
```